In [1]:
# Importing all the libraries needed
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import pandas as pd
import os, requests, cv2, random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import models
from tensorflow.keras import Sequential, layers
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from sklearn.metrics import confusion_matrix,classification_report

In [2]:
# loading the pictures of tomatoes

train_data_dir = 'D:/Chrome Downloads/Tomato/tomato/train'
test_data_dir = 'D:/Chrome Downloads/Tomato/tomato/val' #this folder will be used for evaluating model's perfomance

In [3]:
# for this challenge we are using ImageDataGenerator 

train_datagen = ImageDataGenerator(rescale=1/255.0,
                               shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True,
                                validation_split=0.3)# specifying the validation split inside the function

test_datagen = ImageDataGenerator(rescale=1/255.0,
                                shear_range=0.2,
                                zoom_range=0.2,
                                horizontal_flip=True)

In [4]:
train_gen = train_datagen.flow_from_directory(
                                train_data_dir,
                                target_size=(224, 224),
                                batch_size=32,
                                shuffle=True,
                                class_mode='categorical',
                                subset='training')

Found 7000 images belonging to 10 classes.


In [5]:
val_gen = train_datagen.flow_from_directory(
                                train_data_dir,
                                target_size=(224,224),
                                batch_size=32,
                                shuffle=True,
                                class_mode='categorical',
                                subset='validation')

Found 3000 images belonging to 10 classes.


In [6]:
test_gen = test_datagen.flow_from_directory(
        test_data_dir,
        target_size=(224, 224),
        batch_size=32,
        class_mode='categorical',
        shuffle = False) #shuffle will not affect the accuracy of the model, but will affect the computation of some metrics that depend on the order of the samples

Found 1000 images belonging to 10 classes.


In [7]:
# CNN model layers

cnn = models.Sequential()

cnn.add(layers.Conv2D (32, kernel_size = (3,3), activation='relu', input_shape = [224, 224,3])),
cnn.add(layers.MaxPooling2D(pool_size = (2, 2))),

cnn.add(layers.Conv2D(64, (3,3), activation='relu')),
cnn.add(layers. MaxPooling2D((2, 2))),

cnn.add(layers.Conv2D (64, (3,3), activation='relu')),
cnn.add(layers.MaxPooling2D((2, 2))),

cnn.add(layers.Conv2D(64, (3, 3), activation='relu')), 
cnn.add(layers.MaxPooling2D((2, 2))),

cnn.add(layers. Conv2D (64, (3, 3), activation='relu')), 
cnn.add(layers.MaxPooling2D((2, 2))),

cnn.add(layers.Conv2D(64, (3, 3), activation='relu')), 
cnn.add(layers.MaxPooling2D((2, 2))),

cnn.add(layers.Flatten()),

cnn.add(layers.Dense(64,activation='relu'))
#output layer
cnn.add(layers.Dense(10,activation='softmax'))

In [8]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 64)        0

In [9]:
opt = keras.optimizers.Adam(learning_rate=0.001)

In [10]:
cnn.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

In [11]:
es = EarlyStopping(monitor = 'val_accuracy', 
                   mode = 'max', 
                   patience = 20, 
                   verbose = 1, 
                   restore_best_weights = True)

In [12]:
history = cnn.fit(x = train_gen,
                    callbacks = [es], 
                    steps_per_epoch = 7000/32,
                    epochs = 10,
                    validation_steps = 3000/32,
                    validation_data = val_gen)

Epoch 1/10
218/218 [==============================] - 343s 2s/step - loss: 1.8528 - accuracy: 0.3211 - val_loss: 1.4663 - val_accuracy: 0.4890
Epoch 2/10
218/218 [==============================] - 254s 1s/step - loss: 1.1313 - accuracy: 0.6021 - val_loss: 0.8772 - val_accuracy: 0.6800
Epoch 3/10
218/218 [==============================] - 255s 1s/step - loss: 0.8515 - accuracy: 0.6973 - val_loss: 0.6973 - val_accuracy: 0.7590
Epoch 4/10
218/218 [==============================] - 254s 1s/step - loss: 0.7473 - accuracy: 0.7293 - val_loss: 0.6781 - val_accuracy: 0.7490
Epoch 5/10
218/218 [==============================] - 253s 1s/step - loss: 0.6338 - accuracy: 0.7789 - val_loss: 0.5332 - val_accuracy: 0.8123
Epoch 6/10
218/218 [==============================] - 253s 1s/step - loss: 0.5852 - accuracy: 0.7877 - val_loss: 0.5407 - val_accuracy: 0.8170
Epoch 7/10
218/218 [==============================] - 253s 1s/step - loss: 0.5288 - accuracy: 0.8097 - val_loss: 0.4445 - val_accuracy: 0.8360

In [14]:
scores = cnn.evaluate(test_gen)

32/32 [==============================] - 17s 528ms/step - loss: 0.2588 - accuracy: 0.9360
